In [ ]:
from gradio_client import Client, handle_file   
# импорт contenxtmanager
from contextlib import contextmanager
import time
import os
from pydantic import BaseModel, Field
import csv
from rapidfuzz import process, fuzz
import re

from tqdm import tqdm
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import json
from pathlib import Path

In [34]:
class Timer:
    def __enter__(self):
        self.start = time.time()
        return self  # возвращаем self, чтобы получить доступ к elapsed_time

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        self.elapsed = round(self.end - self.start,4)
        #print(f"Cell execution time: {self.elapsed:.4f} seconds")


In [30]:
from packages.word_corrector.src.core import   PhraseCorrectorByWords, PhraseCorrectorNgrams, correct_text

# Корректор

In [31]:
corrector = PhraseCorrectorByWords("data/words.txt")

In [32]:
to_be_corerected_corpus  = []
to_be_corerected_corpus.append( '''
Пюре Фрутонята
Пюре Фруктоня
Цыпленок 80г
Цыпденок 80r
Мясое изделие для детского питания
''')
to_be_corerected_corpus.append('''
Пюре фруктоня
цыбленок 80000г
мясное для детского питания
''')
to_be_corerected_corpus.append('''
бальзам ШАУМ 7
ТРАВ 300мл 
для нормальных и жирных волос
''')
to_be_corerected_corpus.append('''
Тютюнок Занукелия
500 мг
''')
to_be_corerected_corpus.append('''
Вино АБРАУ-ДЮРОСО
0.75л
13% объёмное, белое п/с
''')

for text in to_be_corerected_corpus:
    print("Исходный текст:\n", text)
    corrected_text, corrections_log = correct_text(text, corrector)
    print("Исправленный текст:\n", corrected_text)
    print("Лог исправлений:\n", corrections_log)
    print("-" * 50)




Исходный текст:
 
Пюре Фрутонята
Пюре Фруктоня
Цыпленок 80г
Цыпденок 80r
Мясое изделие для детского питания

Исправленный текст:
 
ПЮРЕ ФРУТОНЯНЯ
Пюре Фруктоня
Цыпленок 80г
Цыпденок 80r
Мясое изделие для детского питания

Лог исправлений:
 ['', 'ПЮРЕ ФРУТОНЯНЯ', 'Пюре Фруктоня', 'Цыпленок 80г', 'Цыпденок 80r', 'Мясое изделие для детского питания', '']
--------------------------------------------------
Исходный текст:
 
Пюре фруктоня
цыбленок 80000г
мясное для детского питания

Исправленный текст:
 
ПЮРЕ ФРУТОНЯНЯ
цыбленок 80000г
мясное для детского питания

Лог исправлений:
 ['', 'ПЮРЕ ФРУТОНЯНЯ', 'цыбленок 80000г', 'мясное для детского питания', '']
--------------------------------------------------
Исходный текст:
 
бальзам ШАУМ 7
ТРАВ 300мл 
для нормальных и жирных волос

Исправленный текст:
 
БАЛЬЗАМ ШАУМ 7
ТРАВ 300мл 
для нормальных и жирных волос

Лог исправлений:
 ['', 'БАЛЬЗАМ ШАУМ 7', 'ТРАВ 300мл ', 'для нормальных и жирных волос', '']
-----------------------------------------

In [35]:
with Timer() as t:
    time.sleep(1)

print(t.elapsed)

1.0001


In [36]:
DATA_BASE_DIR = Path("/home/jovyan/work/data")

In [37]:
with open(DATA_BASE_DIR / "test_dataset.json", "r") as f:
    test_data = json.load(f)

In [39]:
test_data[:1]

[{'image_path': 'names/930.jpg',
  'text': 'Молоко КРАСНАЯ ЦЕНА\n800мл\nмит.паст.2,5%'}]

In [40]:
client = Client("http://qwen_api:7860/")

Loaded as API: http://qwen_api:7860/ ✔


In [41]:
SYSTEM_PROMPT = '''
Ты система распознавания текста, которая распознает текст с русских ценников в супермаркетах. В тексте могут встречаться названия на английском.
Выведи только текст, построчно, в точности как на изображении. Без дополнительной информации. Не переводи слова на другой язык.
'''

In [42]:
def  predict(file_path, model_name = "Qwen2_5-7b"):    
    result = client.predict(
		system_prompt =SYSTEM_PROMPT,
		user_prompt="",
		image=handle_file(file_path),
        #image=None,
		selected_model=model_name,
		temperature=0.2,
		api_name="/predict"
	)
    return result

In [43]:
predict(DATA_BASE_DIR / "names/1.jpg")

'Вода святой источник\n1.0л\nприродная питьевая негаз.ПЭТ'

In [45]:
class CSVRow(BaseModel):
    file_name: str
    text: str
    elapsed_time: float
    corrected_text: str
    elapsed_time_corrected: float
    original_text:str


In [49]:
def mass_experiment(model_name, limit=1, debug=False, prefix=""):
    with open(f"results_{prefix}{model_name}.tsv", "w", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter="\t", quoting=csv.QUOTE_MINIMAL)
        writer.writerow(CSVRow.model_fields.keys())
    
        for item in tqdm(test_data[:limit], desc="Process images", unit="image"):
            try:
                file_name = item["image_path"]
                original_text = item["text"]
                row = CSVRow(
                    file_name=file_name,
                    text="",
                    elapsed_time=0.0,
                    corrected_text="",
                    elapsed_time_corrected=0.0,
                    original_text=original_text
                )
    
                file_path = DATA_BASE_DIR / file_name
    
                # Чтение изображения
                if debug:
                    try:
                        img = mpimg.imread(file_path)
                        plt.imshow(img)
                        plt.axis('off')
                        plt.show()
                    except Exception as img_error:
                        print(f"[Ошибка изображения] {file_name}: {img_error}")
                        continue  # пропустить эту строку
        
                # Предсказание
                with Timer() as t:
                    result = predict(file_path, model_name)
                row.elapsed_time = t.elapsed
                row.text = result
    
                # Коррекция
                with Timer() as t:
                    corrected_result, _ = correct_text(result, corrector)
                row.elapsed_time_corrected = t.elapsed
                row.corrected_text = corrected_result
    
                # Лог + запись
                if debug:
                    print(json.dumps(row.model_dump(), indent=4, ensure_ascii=False))
                writer.writerow(row.model_dump().values())
                f.flush()
    
            except Exception as e:
                print(f"[Общая ошибка] {item['image_path']}: {e}")
                continue


In [47]:
model_names = [
    "Qwen2_5-3b_finetuned",
    "Qwen2_5-7b_finetuned",
    "Qwen2-vl-7b_finetuned",
    "Qwen2_5-3b",
    "Qwen2_5-7b",
    "Qwen2-vl-7b"
]

# Запуск экспериментов

In [ ]:
for model_name in model_names:
    print(model_name)
    mass_experiment(model_name, -1, False, "wordscorrection_")

Qwen2_5-3b_finetuned


Process images:   2%|▏         | 3/198 [00:26<25:35,  7.87s/image]

[Общая ошибка] names/577.jpg: The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().


Process images:  14%|█▍        | 28/198 [02:37<17:07,  6.04s/image]